In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import math

In [3]:

def conv3x3(in_planes, out_planes, stride=1):
    " 3x3 convolution with padding "
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion=1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet_Cifar(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        super(ResNet_Cifar, self).__init__()
        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x




In [4]:
x = torch.rand(128,3,32,32)
net = ResNet_Cifar(BasicBlock, [3, 3, 3])
begin=time.time()
for i in range(100):
    res1=net(x).data.numpy()
print time.time()-begin

68.3027749062


In [7]:
shrink = 4
class lookupconv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        temp=list(args)
        orig_och=temp[1]
        temp[1]=orig_och/shrink
        args=tuple(temp)
        super(lookupconv, self).__init__(*args, **kwargs)
        self.indices=torch.LongTensor(np.random.choice(orig_och/shrink, orig_och))
    
    def forward(self, input):
        x =super(lookupconv, self).forward(input)
        return torch.index_select(x, 1, self.indices)
    
    
def conv3x3(in_planes, out_planes, stride=1):
    " 3x3 convolution with padding "
    return lookupconv(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion=1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet_Cifar(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        super(ResNet_Cifar, self).__init__()
        self.inplanes = 16
        self.conv1 = lookupconv(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, lookupconv):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                lookupconv(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x




In [8]:
net = ResNet_Cifar(BasicBlock, [3, 3, 3])
begin=time.time()
for i in range(100):
    res1=net(x).data.numpy()
print time.time()-begin

57.871655941


In [8]:
model = ResNet_Cifar(BasicBlock, [3, 3, 3])

In [15]:
list(model.parameters())

[Parameter containing:
 tensor([[[[-0.0287,  0.1122,  0.1536],
           [ 0.1582, -0.0346,  0.1520],
           [ 0.0731,  0.0604,  0.0560]],
 
          [[-0.0563, -0.0582, -0.1059],
           [ 0.1447,  0.1688, -0.0977],
           [ 0.1250,  0.1283,  0.2701]],
 
          [[-0.0505, -0.0222,  0.1057],
           [ 0.2393, -0.0036,  0.1526],
           [-0.1229,  0.0581,  0.2198]]],
 
 
         [[[-0.0241, -0.0401,  0.1158],
           [-0.0519,  0.1266,  0.1147],
           [-0.1488, -0.3112, -0.0003]],
 
          [[-0.0945,  0.3005,  0.0118],
           [-0.0030,  0.1364, -0.0674],
           [ 0.0067,  0.1181,  0.0145]],
 
          [[ 0.0318, -0.0228, -0.2934],
           [ 0.0303,  0.1628, -0.0869],
           [ 0.0823,  0.1649, -0.1492]]],
 
 
         [[[-0.0785,  0.1483,  0.0891],
           [-0.1009, -0.1682,  0.0860],
           [-0.1428, -0.0493,  0.1467]],
 
          [[ 0.0842,  0.0167, -0.2825],
           [-0.0024,  0.2031,  0.1176],
           [-0.0966, -0.0052, 

In [16]:

class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [27]:
x=torch.rand(1,3,224,224)
alexnet=AlexNet()

In [31]:
begin=time.time()
res=alexnet(x).data.numpy()
print time.time()-begin

0.0620849132538


In [32]:
res.shape

(1, 1000)

In [10]:
def test(*args, **kwargs):
    print args

    temp=list(args)
    temp[0]=99
    args=tuple(temp)
    print args
    print kwargs

In [11]:
test(4,a=2)

(4,)
4
(99,)
{'a': 2}
